In [0]:
!pip install -q six numpy scipy matplotlib scikit-image opencv-python imageio
!pip install -q keras imgaug
!pip install -q git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
import keras
from keras_contrib.callbacks import CyclicLR, DeadReluDetector, SnapshotCallbackBuilder, SnapshotModelCheckpoint
import imgaug as ia
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [0]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
# from keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,LambdaCallback
import os,sys,math

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
import google
colab_dir='./'
file_name='EIP_CIFAR_10'
if hasattr(google,'colab'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    colab_dir='/content/gdrive/My Drive/Colab Notebooks/'
model_file=colab_dir+file_name+'.h5'
weights_dir=colab_dir+'weights/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Augmentation and resizing
#augment and then concat samples with original
def augment(dataset,flip=0.5,blur=1.0):
    ia.seed(1)
#     seq = iaa.Sequential([iaa.Fliplr(flip),iaa.GaussianBlur(sigma=(0, blur)),iaa.Sometimes(iaa.Crop(percent=(0, 0.1))),
#                          iaa.Sometimes(iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
#                                              translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
#                                              rotate=(-45, 45),shear=(-16, 16),order=[0, 1],cval=(0, 255),mode=ia.ALL))])
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8))], random_order=True)    
    return seq.augment_images(dataset)

def augmenter(X,y,start=0,end=1):
    na=True
    ln=len(X)
    print('Before augmentation:',X.shape,y.shape)
    start=int(start*ln)
    end=int(end*ln)
    if na:
      X=augment(X)
    else:
      new_X=augment(X)[start:end]
      new_y=y[start:end]
      X=np.concatenate((X,new_X))
      y=np.concatenate((y,new_y))
    print('After augmentation:',X.shape,y.shape)
    return (X,y)

#26x26 is almost half of 32x32. 22x22 maybe too small even though its exact half.
def resize_imgs(imgs,shape=(26,26)):
    seq = iaa.Sequential([iaa.Scale({"height": shape[0], "width": shape[1]})])
    return seq.augment_images(imgs)
 
def load_prev_model(model_small=None):
#     print(os.popen('du -sh '+colab_dir+'*').read())
    last_best=os.popen('du -sh '+colab_dir+r'*|sed -r "s/^[0-9\.]+[MK]?\s(.*)$/\1/g;"|tail -1').read().strip()
    model_prev=None
    try:
        print('Attempting to load last best model from file - ',end='')
        last_best_fn=last_best.split('/')[-1]
        last_best_epoch=last_best_fn.split('.')[1].split('-')[0]
        model_prev=load_model(last_best)
        print('Sucess\nLoaded '+last_best_fn, 'epoch :', last_best_epoch)
    except Exception as e:
        print('Failed!\n',e)
        try:
            print('Attempting to load last saved model from file - ',end='')
            model_prev = load_model(model_file)
            print('Sucess\nLoaded model from file',model_file)
        except Exception as e:
    #         print(str(e), 'at line ', sys.exc_info()[2].tb_lineno)
            print('Failed!\n',e)
            try:
                print('Attempting to load in memory, small model - ',end='')
                if len(model_small.layers)>1:
                    model_prev=model_small
                print('Sucess')
            except Exception as e:
                print('Failed!\n',e)
    return model_prev
  
def copy_weights(model_to,model_from):
    #model_to.set_weights(model_from.get_weights())
    s,err=0,0
    print('Trying to copy weights')
    try:
        if len(model_to.layers) >1 and len(model_from.layers) >1:
            pass
    except Exception as e:
        print('Inavlid models',model_to,model_from)
        return
    for new_layer, layer in zip(model_to.layers[1:], model_from.layers[1:]):
        s+=1
        try:
            new_layer.set_weights(layer.get_weights())
        except Exception as e:
            pass
    print('Done: errors:',err)
      
#new model with larger i/p layer
def larger_model(src_model,shape=(32,32,3)):
    new_input=Input(shape)
    src_model.layers.pop(0)
    new_output=src_model(new_input)
    new_model=Model(new_input,new_output)
    return new_model

# Load CIFAR10 Data
def load_data(resize=False,shape=(26,26),test_augment=False,num_classes=10):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    if resize:
        x_train=resize_imgs(x_train,shape)
        x_test=resize_imgs(x_test,shape)
    
    (x_train, y_train) = augmenter(x_train, y_train,end=1)
    if test_augment:
        (x_test, y_test) = augmenter(x_test, y_test,end=1)
    return (x_train, y_train,x_test, y_test)

In [0]:
# #load data
x_train, y_train, x_test, y_test = load_data(resize=False,shape=(32,32,3))
batch_size=128
#callbacks
model_checkpointer=ModelCheckpoint(weights_dir+'dense_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)

clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))
epochs=60
callbacks = [ model_checkpointer]
hist=[]

Before augmentation: (50000, 32, 32, 3) (50000, 10)


KeyboardInterrupt: ignored

In [0]:
from keras_contrib.applications import DenseNet
# dense_model=DenseNet(input_shape=(32,32,3),depth=100,nb_dense_block=3,growth_rate=12,nb_filter=-1,bottleneck=True,reduction=0.5,dropout_rate=0.5,classes=10,pooling='avg')
dense_model=DenseNet(input_shape=(32,32,3),depth=88,nb_dense_block=4,growth_rate=12,nb_filter=-1,bottleneck=True,reduction=0.5,dropout_rate=0.5,classes=10,pooling='avg')

In [0]:
dense_model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True))
dense_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
dense_0_0_bn (BatchNormalizatio (None, 32, 32, 24)   96          initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           dense_0_0_bn[0][0]               
__________________________________________________________________________________________________
dense_0_0_

In [0]:
h=dense_model.fit(x=x_train, y=y_train, batch_size=batch_size,verbose=1,epochs=epochs, callbacks=callbacks,validation_data=(x_test, y_test))
hist.append(h)
dense_model.save(model_file)
print('Saved dense_model to disk')

In [0]:
# clr = CyclicLR(base_lr=0.001, max_lr=0.01,step_size=8*(len(y_train)/batch_size))
epochs=100
callbacks = [ model_checkpointer]

h=dense_model.fit(x=x_train, y=y_train, batch_size=batch_size,verbose=1,initial_epoch=60,epochs=epochs, callbacks=callbacks,validation_data=(x_test, y_test))
hist.append(h)
dense_model.save(model_file)
print('Saved dense_model to disk')

In [0]:
# Test the model
score = dense_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])